In [2]:
import pandas as pd
import ollama
from urllib.parse import urlparse, parse_qs
from tqdm import tqdm

In [74]:
df=pd.read_csv("result.csv")

In [75]:
df.tail()

,url,type
9995,http://woodmax.in/.usaaupdate/login/,phishing
9996,http://allegro.website/show_item.php=813864719...,phishing
9997,http://domminationsducroixlignes.com/repartiti...,phishing
9998,http://www.bumet.net/data/data/data/,phishing
9999,http://integratedhost.com/update/gmail/login.p...,phishing


In [76]:
# Convert 'type' column (Label Encoding)
df['type'] = df['type'].map({'phishing': 1, 'legitimate': 0})

In [77]:
# Function to tokenize URLs
def tokenize_url(url):
    parsed = urlparse(url)
    return {
        "protocol": parsed.scheme,
        "domain": parsed.netloc,
        "path": parsed.path,
        "num_query_params": len(parse_qs(parsed.query)),
        "has_login": 1 if "login" in parsed.path.lower() or "signin" in parsed.path.lower() else 0,
        "has_ip": 1 if any(char.isdigit() for char in parsed.netloc) else 0,
    }

In [78]:
df_tokens = df['url'].apply(tokenize_url).apply(pd.Series)

# Select key parts for embedding
df['text_for_embedding'] = df_tokens['domain'] + " " + df_tokens['path']


In [79]:
tqdm.pandas()

In [80]:
def get_embedding(text):
    response = ollama.embeddings(model="mistral", prompt=text)
    return response['embedding']

In [81]:
# Apply function with progress bar
df['embedding'] = df['text_for_embedding'].progress_apply(get_embedding)


100%|██████████| 10000/10000 [12:24:12<00:00,  4.47s/it]  


In [82]:
# Convert embeddings into separate columns
embedding_df = pd.DataFrame(df['embedding'].tolist(), index=df.index)
df = pd.concat([df.drop(columns=['embedding', 'text_for_embedding']), df_tokens, embedding_df], axis=1)



In [83]:
# Save Processed Data
df.to_csv("processed_data.csv", index=False)

print("Processing completed. Saved as 'processed_sample.csv'.")

Processing completed. Saved as 'processed_sample.csv'.
